In [59]:
#import all libraries and dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [60]:
excel_file='Prev_result_new1.xlsx'
results = pd.read_excel(excel_file)
results.head()

,date,Team_1,Team_2,Winner,Margin,Ground
0,2012-02-01,Australia,India,Australia,31 runs,ANZ Stadium
1,2012-02-03,Australia,India,India,8 wickets,Melbourne Cricket Ground
2,2012-02-11,New Zealand,Zimbabwe,New Zealand,7 wickets,Eden Park
3,2012-02-14,New Zealand,Zimbabwe,New Zealand,5 wickets,Seddon Park
4,2012-02-17,New Zealand,South Africa,New Zealand,6 wickets,Westpac Stadium


In [61]:
excel_file2='World_cup_gr_new.xls'
world_cup = pd.read_excel(excel_file2)
world_cup.head()


,Team,Group,Previous appearances,Previous titles,Previous finals,Previous semifinals,Current rank
0,West Indies,A,6,2,2,4,9
1,Sri Lanka,A,6,1,3,4,10
2,India,B,6,1,2,3,2
3,Pakistan,B,6,1,2,4,3
4,Australia,A,6,0,1,3,7


In [62]:
df = results[(results['Team_1'] == 'India') | (results['Team_2'] == 'India')]
india = df.iloc[:]

In [63]:

#creating a column for matches played in 2012
year = []
for row in india['date']:
    year.append(int(row[7:]))
india ['match_year']= year
india_2012 = india[india.match_year >= 12]


In [64]:
#narrowing to team patcipating in the world cup
worldcup_teams = ['England', ' South Africa', 'West Indies', 
            'Pakistan', 'New Zealand', 'Sri Lanka', 'Afghanistan', 
            'Australia', 'Bangladesh', 'India']
df_teams_1 = results[results['Team_1'].isin(worldcup_teams)]
df_teams_2 = results[results['Team_2'].isin(worldcup_teams)]
df_teams = pd.concat((df_teams_1, df_teams_2))
df_teams.drop_duplicates()
df_teams.count()

date      53
Team_1    53
Team_2    53
Winner    53
Margin    50
Ground    53
dtype: int64

In [65]:
df_teams.head()

,date,Team_1,Team_2,Winner,Margin,Ground
4,2012-02-17,New Zealand,South Africa,New Zealand,6 wickets,Westpac Stadium
5,2012-02-19,New Zealand,South Africa,South Africa,8 wickets,Seddon Park
7,2012-02-22,New Zealand,South Africa,South Africa,3 runs,Eden Park
41,2012-09-08,England,South Africa,South Africa,7 wickets,Riverside Ground
43,2012-09-10,England,South Africa,England,NaN,Old Trafford


In [66]:
#dropping columns that wll not affect match outcomes
df_teams_2012 = df_teams.drop(['date','Margin', 'Ground'], axis=1)
df_teams_2012.head()

,Team_1,Team_2,Winner
4,New Zealand,South Africa,New Zealand
5,New Zealand,South Africa,South Africa
7,New Zealand,South Africa,South Africa
41,England,South Africa,South Africa
43,England,South Africa,England


In [67]:
#Building the model
#the prediction label: The winning_team column will show "1" Team 1 has won and "2" if the away team has won.

df_teams_2012 = df_teams_2012.reset_index(drop=True)
df_teams_2012.loc[df_teams_2012.Winner == df_teams_2012.Team_1,'winning_team']=1
df_teams_2012.loc[df_teams_2012.Winner == df_teams_2012.Team_2, 'winning_team']=2
df_teams_2012 = df_teams_2012.drop(['winning_team'], axis=1)

df_teams_2012.head()

,Team_1,Team_2,Winner
0,New Zealand,South Africa,New Zealand
1,New Zealand,South Africa,South Africa
2,New Zealand,South Africa,South Africa
3,England,South Africa,South Africa
4,England,South Africa,England


In [68]:
# Get dummy variables
final = pd.get_dummies(df_teams_2012, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

# Separate X and y sets
X = final.drop(['Winner'], axis=1)
y = final["Winner"]


# Separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [69]:

final.head()

,Winner,Team_1_Afghanistan,Team_1_Australia,Team_1_Bangladesh,Team_1_England,Team_1_India,Team_1_Ireland,Team_1_Netherlands,Team_1_New Zealand,Team_1_Pakistan,Team_1_South Africa,Team_1_Sri Lanka,Team_1_West Indies,Team_2_ South Africa,Team_2_Australia
0,New Zealand,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,South Africa,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,South Africa,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,South Africa,0,0,0,1,0,0,0,0,0,0,0,0,1,0
4,England,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [70]:
#Training the model by using Ranom forest Algorithm
rf = RandomForestClassifier(n_estimators=150, max_depth=20,
                              random_state=0)
rf.fit(X_train, y_train) 


score = rf.score(X_train, y_train)
score2 = rf.score(X_test, y_test)


print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))


Training set accuracy:  0.667
Test set accuracy:  0.455


In [71]:

#Training the model by using SVM Algorithm
from sklearn import svm
svm=svm.SVC(gamma='scale')
svm.fit(X_train, y_train)
score=svm.score(X_train, y_train)
score2=svm.score(X_train, y_train)

print("Training set accuracy_svm:",'%.3f'%(score))
print("Test set accuracy_svm:",'%.3f'%(score2))

Training set accuracy_svm: 0.667
Test set accuracy_svm: 0.667


In [72]:
#adding ICC rankings
#the team which is positioned higher on the ICC Ranking will be considered "favourite" for the match
#and therefore, will be positioned under the "Team_1" column

# Loading new datasets
excel_file3='Ranking_new.xls'
ranking = pd.read_excel(excel_file3)
excel_file4='Fixtures_new.xlsx'
fixtures= pd.read_excel(excel_file4)


# List for storing the group stage games
pred_set = []



In [73]:
# Create new columns with ranking position of each team
fixtures.insert(1, 'first_position', fixtures['Team_1'].map(ranking.set_index('Team')['Position']))
fixtures.insert(2, 'second_position', fixtures['Team_2'].map(ranking.set_index('Team')['Position']))

# We only need the group stage games, so we have to slice the dataset
fixtures = fixtures.iloc[ :21, :]
fixtures.tail()

,Round Number,first_position,second_position,Date,Location,Team_1,Team_2,Group,Result
16,1,7.0,6.0,2021-11-04,Dubai,Australia,Bangladesh,A,NaN
17,1,9.0,10.0,2021-11-04,Abu dhabi,West Indies,Sri Lanka,A,NaN
18,1,7.0,9.0,2021-11-06,Abu dhabi,Australia,West Indies,A,NaN
19,1,1.0,5.0,2021-11-06,Sharjah,England,South Africa,A,NaN
20,1,4.0,8.0,2021-11-07,Abu dhabi,New Zealand,Afghanistan,B,NaN


In [74]:
for index, row in fixtures.iterrows():
    if row['first_position'] < row['second_position']:
        pred_set.append({'Team_1': row['Team_1'], 'Team_2': row['Team_2'], 'winning_team': None})
    else:
        pred_set.append({'Team_1': row['Team_2'], 'Team_2': row['Team_1'], 'winning_team': None})
        
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set
pred_set.head()

,Team_1,Team_2,winning_team
0,South Africa,Australia,None
1,England,West Indies,None
2,Bangladesh,Sri Lanka,None
3,India,Pakistan,None
4,Pakistan,New Zealand,None


In [75]:
pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

# Add missing columns compared to the model's training dataset
missing_cols = set(final.columns) - set(pred_set.columns)
for c in missing_cols:
    pred_set[c] = 0
pred_set = pred_set[final.columns]


pred_set = pred_set.drop(['Winner'], axis=1)
pred_set.head()

,Team_1_Afghanistan,Team_1_Australia,Team_1_Bangladesh,Team_1_England,Team_1_India,Team_1_Ireland,Team_1_Netherlands,Team_1_New Zealand,Team_1_Pakistan,Team_1_South Africa,Team_1_Sri Lanka,Team_1_West Indies,Team_2_ South Africa,Team_2_Australia
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [76]:
#Predictions for super 12 Teams by using Support Vector Machine
predictions = svm.predict(pred_set)
for i in range(fixtures.shape[0]):
    print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
    if predictions[i] == 1:
        print("Winner: " + backup_pred_set.iloc[i, 1])
    
    else:
        print("Winner: " + backup_pred_set.iloc[i, 0])
    print("")

Australia and South Africa
Winner: South Africa

West Indies and England
Winner: England

Sri Lanka and Bangladesh
Winner: Bangladesh

Pakistan and India
Winner: India

New Zealand and Pakistan
Winner: Pakistan

West Indies and South Africa
Winner: South Africa

Bangladesh and England
Winner: England

Sri Lanka and Australia
Winner: Australia

Afghanistan and Pakistan
Winner: Pakistan

West Indies and Bangladesh
Winner: Bangladesh

Sri Lanka and South Africa
Winner: South Africa

Australia and England
Winner: England

New Zealand and India
Winner: India

Sri Lanka and England
Winner: England

Bangladesh and South Africa
Winner: South Africa

Afghanistan and India
Winner: India

Australia and Bangladesh
Winner: Bangladesh

Sri Lanka and West Indies
Winner: West Indies

West Indies and Australia
Winner: Australia

South Africa and England
Winner: England

Afghanistan and New Zealand
Winner: New Zealand



In [77]:
#Predictions for super 12 Teams by using Random Forest
predictions = rf.predict(pred_set)
for i in range(fixtures.shape[0]):
    print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
    if predictions[i] == 1:
        print("Winner: " + backup_pred_set.iloc[i, 1])
    
    else:
        print("Winner: " + backup_pred_set.iloc[i, 0])
    print("")

Australia and South Africa
Winner: South Africa

West Indies and England
Winner: England

Sri Lanka and Bangladesh
Winner: Bangladesh

Pakistan and India
Winner: India

New Zealand and Pakistan
Winner: Pakistan

West Indies and South Africa
Winner: South Africa

Bangladesh and England
Winner: England

Sri Lanka and Australia
Winner: Australia

Afghanistan and Pakistan
Winner: Pakistan

West Indies and Bangladesh
Winner: Bangladesh

Sri Lanka and South Africa
Winner: South Africa

Australia and England
Winner: England

New Zealand and India
Winner: India

Sri Lanka and England
Winner: England

Bangladesh and South Africa
Winner: South Africa

Afghanistan and India
Winner: India

Australia and Bangladesh
Winner: Bangladesh

Sri Lanka and West Indies
Winner: West Indies

West Indies and Australia
Winner: Australia

South Africa and England
Winner: England

Afghanistan and New Zealand
Winner: New Zealand



In [78]:
# List of tuples before 
semi = [('England', 'Pakistan'),
            ('India','South Africa' )]

In [79]:
def clean_and_predict(matches, ranking, final, logreg):

    # Initialization of auxiliary list for data cleaning
    positions = []

    # Loop to retrieve each team's position according to ICC ranking
    for match in matches:
        positions.append(ranking.loc[ranking['Team'] == match[0],'Position'].iloc[0])
        positions.append(ranking.loc[ranking['Team'] == match[1],'Position'].iloc[0])
    
    # Creating the DataFrame for prediction
    pred_set = []

    # Initializing iterators for while loop
    i = 0
    j = 0

    # 'i' will be the iterator for the 'positions' list, and 'j' for the list of matches (list of tuples)
    while i < len(positions):
        dict1 = {}

        # If position of first team is better then this team will be the 'Team_1' team, and vice-versa
        if positions[i] < positions[i + 1]:
            dict1.update({'Team_1': matches[j][0], 'Team_2': matches[j][1]})
        else:
            dict1.update({'Team_1': matches[j][1], 'Team_2': matches[j][0]})

        # Append updated dictionary to the list, that will later be converted into a DataFrame
        pred_set.append(dict1)
        i += 2
        j += 1
        # Convert list into DataFrame
    pred_set = pd.DataFrame(pred_set)
    backup_pred_set = pred_set

    # Get dummy variables and drop winning_team column
    pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

    # Add missing columns compared to the model's training dataset
    missing_cols2 = set(final.columns) - set(pred_set.columns)
    for c in missing_cols2:
        pred_set[c] = 0
    pred_set = pred_set[final.columns]

    pred_set = pred_set.drop(['Winner'], axis=1)

    # Predict!
    predictions = logreg.predict(pred_set)
    for i in range(len(pred_set)):
        print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
        if predictions[i] == 1:
            print("Winner: " + backup_pred_set.iloc[i, 1])
        else:
            print("Winner: " + backup_pred_set.iloc[i, 0])
        print("")

In [80]:
#Predictions for Semi-finals by using SVM
clean_and_predict(semi, ranking, final, svm)

Pakistan and England
Winner: England

South Africa and India
Winner: India



In [81]:
#Predictions for Semi-finals by using rf
clean_and_predict(semi, ranking, final, rf)

Pakistan and England
Winner: England

South Africa and India
Winner: India



In [82]:
# Finals
finals = [('India', 'England')]

In [30]:
#Predictions for Finals by using SVM
clean_and_predict(finals, ranking, final, svm)

India and England
Winner: England



In [31]:
#Predictions for Finals by using rf
clean_and_predict(finals, ranking, final, rf)

India and England
Winner: England

